# DAS Pattern Matching Queries - Animals Dataset

This notebook demonstrates how to use `hyperon-experimental` with the Distributed Atomspace (DAS) to perform pattern matching queries. We use a [simple animals knowledge base](https://raw.githubusercontent.com/singnet/das/refs/heads/master/src/tests/integration/cpp/data/animal_expressions.metta) as example.

## Setup

Initialize `hyperon` MeTTa environment and create a helper function to run MeTTa programs.


In [1]:
import hyperon

metta = hyperon.MeTTa()
def run(program='!(+ 1 2)'):
    for result in metta.run(program):
        for child in result:
            print(child)

## Import DAS Module

Import the DAS module into the MeTTa environment.


In [2]:
run('!(import! &self das)')

()


## Connect to DAS

Bind a DAS connection to `&das` space. The first parameter specifies a client's host and port range (47000-47999) and the second must be a known peer address (eg. Query Agent at localhost:40002).

For `MacOS` users you will need to use `host.docker.internal` as this notebook host (eg. `host.docker.internal:47000-47999`)


In [3]:
run('!(bind! &das (new-das! (localhost:47000-47999) (localhost:40002)))')

()


## Query 1: Simple Pattern Match

Find all evaluations in the knowledge base. Variables `$P` (predicate) and `$C` (concept) will match any predicate and concept.


In [4]:
run('!(match &das (Evaluation $P $C) (Evaluation $P $C))')

(Evaluation (Predicate "is_animal") (Concept "chimp"))
(Evaluation (Predicate "no_legs") (Concept "earthworm"))
(Evaluation (Predicate "is_mammal") (Concept "rhino"))
(Evaluation (Predicate "is_reptile") (Concept "triceratops"))
(Evaluation (Predicate "is_plant") (Concept "vine"))
(Evaluation (Predicate "four_legs") (Concept "rhino"))
(Evaluation (Predicate "has_horns") (Concept "rhino"))
(Evaluation (Predicate "is_dinosaur") (Concept "snake"))
(Evaluation (Predicate "four_legs") (Concept "triceratops"))
(Evaluation (Predicate "has_horns") (Concept "triceratops"))
(Evaluation (Predicate "is_plant") (Concept "ent"))
(Evaluation (Predicate "is_animal") (Concept "ent"))
(Evaluation (Predicate "is_mammal") (Concept "monkey"))
(Evaluation (Predicate "is_animal") (Concept "earthworm"))
(Evaluation (Predicate "two_legs") (Concept "chimp"))
(Evaluation (Predicate "is_animal") (Concept "snake"))
(Evaluation (Predicate "is_reptile") (Concept "snake"))
(Evaluation (Predicate "is_animal") (Concept

## Query 2: OR Pattern

Find concepts that are either animals OR have two legs. This demonstrates disjunctive queries.


In [5]:
run('''
  !(match 
        &das 
        (or 
          (Evaluation (Predicate "is_animal") (Concept $C)) 
          (Evaluation (Predicate "two_legs") (Concept $C))) 
        $C)
''')

"human"
"triceratops"
"monkey"
"chimp"
"earthworm"
"snake"
"rhino"
"ent"


## Query 3: AND Pattern

Find concepts that are both mammals AND have four legs.


In [6]:
run('''
  !(match 
        &das 
        (and 
           (Evaluation (Predicate "is_mammal") (Concept $C)) 
           (Evaluation (Predicate "four_legs") (Concept $C))) 
        $C)
''')

"rhino"


## Query 4: Nested AND Pattern

Complex nested query to find concepts that are: animals AND reptiles AND dinosaurs AND have horns AND have four legs.


In [7]:
run('''
  !(match 
        &das
        (and
            (and
                (Evaluation (Predicate "is_animal") (Concept $C))
                (Evaluation (Predicate "is_reptile") (Concept $C)))
            (and
                (Evaluation (Predicate "is_dinosaur") (Concept $C))
                (and
                    (Evaluation (Predicate "has_horns") (Concept $C))
                    (Evaluation (Predicate "four_legs") (Concept $C)))))
        $C)
''')

"triceratops"


## Query 5: Reverse Lookup

Find all predicates that apply to a specific concept (triceratops). This shows all properties of "triceratops".


In [8]:
run('!(match &das (Evaluation (Predicate $P) (Concept "triceratops")) $P)')

"is_reptile"
"four_legs"
"has_horns"
"is_animal"
"is_dinosaur"


## Query 6: Nested AND and OR Pattern

Find all concepts that are classified as plants.


In [9]:
run('''
  !(match 
        &das 
        (and
          (Evaluation (Predicate "is_animal") (Concept $C)) 
          (or
             (Evaluation (Predicate "two_legs") (Concept $C))
             (Evaluation (Predicate "no_legs") (Concept $C))))
        $C)
''')

"chimp"
"human"
"monkey"
"earthworm"
